#### This notebook explains the Abstract Syntax Tree (AST). 

It parses a simple method to show the structure of its nodes, then implements a basic RuboCop-style cop to demonstrate how it works.

This tutorial walks you through the process:

- [Static vs. Dynamic Code Analysis: How RuboCop Reads Ruby’s Mind](https://medium.com/jungletronics/static-vs-dynamic-code-analysis-how-rubocop-reads-rubys-mind-c3e190a28420)

- Explore the key differences between static and dynamic analysis — and discover how RuboCop inspects your code’s Abstract Syntax Tree (AST) to catch issues before your program even runs.

Enjoy!



In [18]:
`rails -v`

"Rails 8.0.4\n"

In [19]:
`ruby -v`

"ruby 3.4.2 (2025-02-15 revision d2930f8e7a) +PRISM [x86_64-linux]\n"

### Method Under Test

In [20]:
def 
  greet(name) 
  puts "Hello, #{name}" 
end
greet('Gilberto')

Hello, Gilberto


### AST

In [21]:
require 'ripper'
require 'pp'

code = <<~RUBY
  def greet(name)
    puts "Hello, \#{name}"
  end
RUBY

"def greet(name)\n  puts \"Hello, \#{name}\"\nend\n"

In [22]:
pp Ripper.sexp(code)

[:program,
 [[:def,
   [:@ident, "greet", [1, 4]],
   [:paren,
    [:params, [[:@ident, "name", [1, 10]]], nil, nil, nil, nil, nil, nil]],
   [:bodystmt,
    [[:command,
      [:@ident, "puts", [2, 2]],
      [:args_add_block,
       [[:string_literal,
         [:string_content,
          [:@tstring_content, "Hello, ", [2, 8]],
          [:string_embexpr, [[:var_ref, [:@ident, "name", [2, 17]]]]]]]],
       false]]],
    nil,
    nil,
    nil]]]]


[:program, [[:def, [:@ident, "greet", [1, 4]], [:paren, [:params, [[:@ident, "name", [1, 10]]], nil, nil, nil, nil, nil, nil]], [:bodystmt, [[:command, [:@ident, "puts", [2, 2]], [:args_add_block, [[:string_literal, [:string_content, [:@tstring_content, "Hello, ", [2, 8]], [:string_embexpr, [[:var_ref, [:@ident, "name", [2, 17]]]]]]]], false]]], nil, nil, nil]]]]

## Let's Practice!

Here is a mini-cop to print offenses if found, or “No offenses” if everything is fine.

In [23]:
require "parser/ruby34"  # loads parser for Ruby 3.4.x

def check_double_quotes(code)
  parser = Parser::CurrentRuby.new
  buffer = Parser::Source::Buffer.new('(example)')
  buffer.source = code
  ast = parser.parse(buffer)

  offenses = []

  check_node = lambda do |node|
    return unless node.is_a?(Parser::AST::Node)

    node.children.each { |child| check_node.call(child) if child.is_a?(Parser::AST::Node) }

    if node.type == :str
      source = node.location.expression.source
      if source.start_with?('"') && !source.include?('#{')
        offenses << "⚠️  Offense at line #{node.location.line}: Prefer single quotes for #{source}"
      end
    end
  end

  check_node.call(ast)

  if offenses.empty?
    puts "✅ No offenses detected!"
  else
    puts offenses
  end
end


:check_double_quotes

#### RUNNING

In [24]:
code1 = <<~RUBY
  name = "Gilberto"
  puts "Hello, \#{name}"
  puts "Static analysis is cool"
RUBY

"name = \"Gilberto\"\nputs \"Hello, \#{name}\"\nputs \"Static analysis is cool\"\n"

In [25]:
check_double_quotes(code1)

⚠️  Offense at line 1: Prefer single quotes for "Gilberto"
⚠️  Offense at line 3: Prefer single quotes for "Static analysis is cool"


In [26]:
code2 = <<~RUBY
  name = 'Gilberto'
  puts "Hello, \#{name}"
  puts 'Static analysis is cool'
RUBY

"name = 'Gilberto'\nputs \"Hello, \#{name}\"\nputs 'Static analysis is cool'\n"

In [27]:
check_double_quotes(code2)

✅ No offenses detected!


#### Summary 

This lambda is a recursive AST walker:

> Checks if the node is valid.

> Recursively visits all children nodes.

When it finds a string literal:

> Checks if it violates the “single quotes preferred” rule.

> Records it in offenses.

### Let’s get a better understanding of the AST!

In [28]:
`ruby-parse -v`

"ruby-parse based on parser version 3.3.10.0\n"

## Lets you see how Ruby interprets code internally.
### AST (Abstract Syntax Tree) - RoboCop Parser

Useful for writing linters, analyzers, or code transformations, like what RuboCop does.

Helps you understand method calls, operators, and literals at the AST level.

In [29]:
result = `ruby-parse -L -e "2+2"`
puts result


s(:send,
  s(:int, 2), :+,
  s(:int, 2))
2+2
 ~ selector   
~~~ expression
s(:int, 2)
2+2
~ expression
s(:int, 2)
2+2
  ~ expression


### Command:
`ruby-parse -L -e "2+2"`

__ruby-parse__ is part of the __parser gem__. It __parses__ Ruby code and __outputs__ its __AST (Abstract Syntax Tree)__.

`-L` enables location information for expressions in the AST.

`-e "2+2"` tells it to parse the code `"2+2"` from the command line.

### Result: 
```
s(:send,
  s(:int, 2), :+,
  s(:int, 2))
```

#### This is the AST (Abstract Syntax Tree) representation of 2+2:

`s(...)` → AST node constructor.

`:send` → method call node (sending the + message to the left-hand side).

`s(:int, 2)` → integer literal node with value 2.

`:+` → the method being called (here, addition).

So this structure says:

`“Call the + method on integer 2 with argument 2.”`


`2+2` → the source code.

`~ selector` → the operator +.

`~~~ expression` → spans the whole node’s expression.

`s(:int, 2)` → integer literals in the AST.

It’s basically a visual mapping between the AST nodes and the original source code.

In [30]:
`rspec --init`

"   exist   .rspec\n   exist   spec/spec_helper.rb\n"

In [31]:
# Get current working directory
puts Dir.pwd

/home/gilberto


Thank You!